### Imports

In [1]:
import pandas as pd
import polars as pl
import numpy as np
#import matplotlib.pyplot as plt
#from feature_baggingV2 import FeatureBaggingWithHyperparamTuning
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
#import lightgbm as lgb
import datetime
from sklearn.preprocessing import RobustScaler
#from keras.models import Sequential
#from keras.layers import LSTM, Dense
import labolibrary as labo

import os, sys, gc, time, warnings, pickle, psutil, random

import warnings
warnings.filterwarnings("ignore")


from sklearn.model_selection import train_test_split
from sktime.clustering.k_means import TimeSeriesKMeans
from sktime.clustering.utils.plotting._plot_partitions import plot_cluster_algorithm
from sktime.datasets import load_arrow_head

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


### Función para escalar/desescalar y métrica

In [2]:
#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'

# Leer datos
df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-PCA-Decompose.parquet') 
df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)


In [3]:
# Ensure periodo is not the index for fitting the model
df_final.reset_index(inplace=True)
df_final['cluster'].fillna(99, inplace=True)
# Create a pivot table to reshape the dataframe
pivot_df = df_final.pivot(index='periodo', columns=['product_id', 'customer_id','cluster'], values='tn')

pivot_df.fillna(0, inplace=True)

In [4]:

# Transpose and reshape the pivot table to get the right format for clustering
ts_data = pivot_df.T.values.reshape(pivot_df.shape[1], pivot_df.shape[0], 1)



model = TimeSeriesKMeans(n_clusters=50, metric="dtw",
                        init_algorithm = 'kmeans++',
                        n_init=2, max_iter=200, tol=10000,
                        verbose = True,
                        random_state=42
                        )

# Fit the model and predict clusters
clusters_dtw = model.fit_predict(ts_data)

# Create a DataFrame for the clusters with the same index as the pivot table columns
cluster_df = pd.DataFrame({'dtw_cluster': clusters_dtw}, index=pivot_df.columns)


Iteration 0, inertia 22545448.78392304.
Iteration 1, inertia 19747027.853266224.
Iteration 2, inertia 19483644.25306578.
Iteration 3, inertia 19302297.759698823.
Iteration 4, inertia 19183771.720519777.
Iteration 5, inertia 19075895.201405022.
Iteration 6, inertia 18935101.337240875.
Iteration 7, inertia 18824707.778399542.
Iteration 8, inertia 18754459.65610337.
Iteration 9, inertia 18709384.996970456.
Iteration 10, inertia 18679629.087549094.
Iteration 11, inertia 18661397.47500557.
Iteration 12, inertia 18649965.650034938.
Iteration 0, inertia 24691609.505517684.
Iteration 1, inertia 19777576.11766885.
Iteration 2, inertia 19532052.269500513.
Iteration 3, inertia 19403515.426485643.
Iteration 4, inertia 19334307.335184302.
Iteration 5, inertia 19266901.39642516.
Iteration 6, inertia 19205949.432534423.
Iteration 7, inertia 19143203.092805266.
Iteration 8, inertia 19087276.68419864.
Iteration 9, inertia 19015527.38019772.
Iteration 10, inertia 18923985.517054982.
Iteration 11, inerti

In [5]:
cluster_df.reset_index(inplace=True)

In [6]:
df_final = df_final.merge(cluster_df, on=['product_id', 'customer_id','cluster'], how='left')

In [7]:

df_final.set_index('periodo', inplace=True)

In [8]:
df_final.to_parquet(DATOS_DIR+'FE_dataset-PCA-Decompose-Cluster-DTW.parquet')

In [9]:
import gc
gc.collect()

0

In [10]:

df_final = df_final.loc['2018-01-01':'2019-10-01']
df_final.groupby(['dtw_cluster']).size()

dtw_cluster
0     2949223
1           3
2          22
3          22
4          22
5          22
6          22
7          22
8          22
9          22
10         22
11         22
12         22
13         22
14         22
15         22
16         22
17         22
18         44
19         66
20     137445
21         22
22         22
23          7
24         14
25         22
26       1890
27         22
28         22
29         20
30         22
31         22
32        176
33         66
34         22
35         22
36         44
37         88
38         22
39         22
40         14
41        132
42         22
43         22
44         44
45       4657
46         44
47         18
48         22
49        782
dtype: int64